In [ ]:
import os

In [ ]:
# clean last run result

!('./clean.sh')

In [ ]:
# add the package to import path

import sys 
sys.path.append("../../")

In [ ]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase

### parameters for Si crystal

In [ ]:
from ase.build import bulk

In [ ]:
atoms = bulk('Si', 'diamond', a=5.459)

In [ ]:
# set Fdata dir
Fdata_path='~/Fdata/Fdata-McWEDA-0.15-3SN.Sis4.8p5.35/'

### parameters for fireball

In [ ]:
kwargs = {'kpt_size': [5, 5, 5],
          }

### Run Fireball

In [ ]:
calc = Fireball(command='fireball-ase.3.x', 
                Fdata_path=Fdata_path,
                **kwargs)
atoms.set_calculator(calc)

In [ ]:
# Running this line will take about 1 min
e0 = atoms.get_potential_energy()
efermi = atoms.calc.get_fermi_level()

print("The energy is {:.3f} eV.".format(e0))
print("The Fermi Level is {:.3f} eV.".format(efermi))

### DOS plot

In [ ]:
from ase.dft import DOS

In [ ]:
dos = DOS(calc, width=0.2, window=[-8,4], npts=1000)

In [ ]:
d = dos.get_dos()
e = dos.get_energies()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(e, d)
plt.xlabel('energy [eV]')
plt.ylabel('DOS')

### Band structure calculation

In [ ]:
lat = atoms.cell.get_bravais_lattice()
print(list(lat.get_special_points()))

In [ ]:
path = lat.bandpath(path="LGXU,KG",npoints=200)

In [ ]:
print(path.path)

In [ ]:
kwargs = {'kpt_path': path,  # VIP for band structure calculation.
          'taurelax': 5.0,
          'efermi_T': 200.0,
          'ifix_CHARGES': 1,  # VIP, fix previous charges
          'beta_set': 0.04,
          }

In [ ]:
calc = Fireball(command='/home/ren/bin/lightning.3.x', 
                Fdata_path=Fdata_path,
                **kwargs)
atoms.set_calculator(calc)

In [ ]:
bs = atoms.calc.band_structure(atoms, reference=efermi) # Note: should use the scf fermi level

In [ ]:
bs.subtract_reference().plot(emin=-15, emax=10)

Ref: https://commons.wikimedia.org/wiki/File:Band_structure_Si_schematic.svg

![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Band_structure_Si_schematic.svg/580px-Band_structure_Si_schematic.svg.png)

Ref: http://lampx.tugraz.at/~hadley/ss1/semiconductors/Si_bandstructure.png

![](http://lampx.tugraz.at/~hadley/ss1/semiconductors/Si_bandstructure.png)